# Mask Detector

Copyright @ 2020 **ABCOM Information Systems Pvt. Ltd.** All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

## Importing modules

In [1]:
import tensorflow 
from tensorflow import keras 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import os


## Loading dataset

In [2]:
!wget https://github.com/abcom-mltutorials/mask_detector/archive/master.zip

--2020-12-10 06:58:37--  https://github.com/abcom-mltutorials/mask_detector/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/abcom-mltutorials/mask_detector/zip/master [following]
--2020-12-10 06:58:37--  https://codeload.github.com/abcom-mltutorials/mask_detector/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [        <=>         ]  41.52M  29.5MB/s    in 1.4s    

2020-12-10 06:58:39 (29.5 MB/s) - ‘master.zip’ saved [43535909]



In [3]:
! unzip master.zip

Archive:  master.zip
b1b44b610cdf1cf28c412f574862ffcaee1c54ed
   creating: mask_detector-master/
   creating: mask_detector-master/data/
   creating: mask_detector-master/data/with_mask/
  inflating: mask_detector-master/data/with_mask/masked_1,000.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,001.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,002.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,003.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,004.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,005.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,006.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,007.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,008.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,009.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,010.jpg  
  inflating: mask_detector-master/data/with_mask/masked_1,011.jpg

# Organizing data

In [4]:
# define path and catogories for classification
path_dir = "/content/mask_detector-master/data"
categories = ["with_mask", "without_mask"]

## Load images 

In [5]:
print("Loading the images")
# make arrays for storing dataset of images and it's label of 
# with mask and wihtout mask
dataset = []
label = []
# join path for preprocessing of images
for category in categories:
    path = os.path.join(path_dir, category)
    for img in os.listdir(path):
      img_path = os.path.join(path, img)
      #print (img_path)
      Img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
      Img = keras.preprocessing.image.img_to_array(Img)
      Img = keras.applications.mobilenet_v2.preprocess_input(Img)
      
      dataset.append(Img)
      label.append(category)

print("All images loaded successfully")

Loading the images
All images loaded successfully


## Convert categories into binary format

In [6]:
# Convert two categories into binary format like 0,1
label_binarizer = LabelBinarizer()
# Transform multi-class labels to binary labels
label = label_binarizer.fit_transform(label)
# Converts a class vector (integers) to binary class matrix
label = keras.utils.to_categorical(label)

## Create arrays of dataset and labels

In [7]:
dataset = np.array(dataset, dtype="float32")
label = np.array(label)

## Split dataset into training and testing

In [8]:
(trainX, testX, trainY, testY) = train_test_split(dataset, label,
	test_size=0.20, stratify=label, random_state=42)

## Apply augmentation on images

In [9]:
#use augmentation to increase variation in dataset by various operation like shifting and rotating images
data_augmentation = keras.preprocessing.image.ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.07,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.10, 
	horizontal_flip=True,
	fill_mode="nearest")

#Creating model

## Load pretrained mobilenetv2 model

In [10]:
# input_shape is shape of image(height, width, channeli.e. 3 here RGB)
# include_top is for fully connected layer
# imagenet is weights from pre-trained model
pre_trained_model = keras.applications.MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=keras.layers.Input(shape=(224, 224, 3)))

# we will freeze the base model to prevent them from updation while training
for layer in pre_trained_model.layers:
	layer.trainable = False


9412608/9406464 [==============================] - 0s 0us/step


## Sequential model

In [11]:
model = tensorflow.keras.Sequential(
    [keras.Input(shape=(224, 224, 3)),
     pre_trained_model,
     keras.layers.MaxPooling2D(pool_size=(2,2)),
     keras.layers.Dense(720, activation="relu"),
     keras.layers.Dense(32, activation="relu"),
     keras.layers.Dropout(0.3),
     keras.layers.Flatten(),
     keras.layers.Dense(units=2,activation="softmax")
     ] 
) 



##Build the model 

In [12]:
model.build([None, 224, 224, 3])
#print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 1280)        0         
_________________________________________________________________
dense (Dense)                (None, 3, 3, 720)         922320    
_________________________________________________________________
dense_1 (Dense)              (None, 3, 3, 32)          23072     
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 5

## Compiling the model

In [13]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

## Train the model

In [14]:
# train the head of the network
EPOCHS =10
batchSize = 40
FIT = model.fit(
	data_augmentation.flow(trainX, trainY, batch_size=batchSize),
	steps_per_epoch=len(trainX) // batchSize,
	validation_data=(testX, testY),
	validation_steps=len(testX) // batchSize,
	epochs= EPOCHS)


Epoch 1/10
85/85 [==============================] - 36s 425ms/step - loss: 0.1364 - accuracy: 0.9571 - val_loss: 0.1372 - val_accuracy: 0.9708
Epoch 2/10
85/85 [==============================] - 35s 410ms/step - loss: 0.0518 - accuracy: 0.9846 - val_loss: 0.0790 - val_accuracy: 0.9883
Epoch 3/10
85/85 [==============================] - 35s 411ms/step - loss: 0.0340 - accuracy: 0.9887 - val_loss: 0.0578 - val_accuracy: 0.9860
Epoch 4/10
85/85 [==============================] - 35s 411ms/step - loss: 0.0450 - accuracy: 0.9879 - val_loss: 0.1218 - val_accuracy: 0.9754
Epoch 5/10
85/85 [==============================] - 35s 414ms/step - loss: 0.0462 - accuracy: 0.9867 - val_loss: 0.0491 - val_accuracy: 0.9883
Epoch 6/10
85/85 [==============================] - 35s 414ms/step - loss: 0.0223 - accuracy: 0.9926 - val_loss: 0.0589 - val_accuracy: 0.9883
Epoch 7/10
85/85 [==============================] - 35s 416ms/step - loss: 0.0294 - accuracy: 0.9905 - val_loss: 0.0684 - val_accuracy: 0.9871

## Prediction of model

In [15]:
PREDICT = model.predict(testX, batch_size=batchSize)

## Check test batch accuracy

In [16]:
# Make 2 lists to contain max values for predicted and actual test values
pred_class = []
ac_class = []
for i in range(len(PREDICT)):
    pr = PREDICT[i].argmax()
    pred_class.append(pr)
    ac = testY[i].argmax()
    ac_class.append(ac)

In [17]:
# Make 2 lists to contain the predicted and actual output results gained from 
# comparing max values in last step with alotted strings     
output = {0:"with_mask", 1:"wihout_mask"}
pred_op = []
ac_op = []
for i in range(len(pred_class)):
   pred_op.append(output[pred_class[i]])
   ac_op.append(output[ac_class[i]])

In [18]:
# Increment counter if the actual and predicted values match. Then calculate 
# % accuracy of the first batch.
correct_pred = 0
for i in range(batchSize):
    if pred_op[i] == ac_op[i]:
        correct_pred += 1

print("Accuracy of batch = ", (correct_pred/batchSize)*100, "%")

Accuracy of batch =  97.5 %


#Saving the model

In [19]:
model.save("mask_detector.model", save_format="h5")
print("The model is saved successfully!! Try detection by running the warn file.")

The model is saved successfully!! Try detection by running the warn file.
